# JSON-LD Conversions

This notebook demonstrates to convert Resources to JSON-LD and vice-versa, and the different possibilities to asign a context to a Resource. Thus has the following priority:
1. use the context explicitly assigned to the resource i.e.`jane.context="http://shchema.org"`
2. use the context configured in the RdfModel if any
3. Let the store add a context (if it is a RDF-store)

## Initialisation

Run the [Blue Brain Nexus project creation notebook](https://github.com/BlueBrain/nexus-forge/blob/master/examples/notebooks/nexus-demo/00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/v1) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

# Context

In [ ]:
context = {
    "ex": "http://example.org/",
    "Person": "ex:Person",
    "Organization": "ex:Organization",
    "employer": "ex:employer",
    "name": "ex:name"
}

# Imports

In [ ]:
import json

In [ ]:
from kgforge.core import Resource

In [ ]:
def pp(x): 
    if x is not None:
        print(json.dumps(x, indent=4)) 

## Resource to JSON-LD

### context in the Resource

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
jane.context = context

In [ ]:
print(jane)

In [ ]:
forge.register(jane)

In [ ]:
pp(forge.as_jsonld(jane, form="compacted"))

In [ ]:
 pp(forge.as_jsonld(jane))

In [ ]:
pp(forge.as_jsonld(jane, store_metadata=True))

### context available in the web

In [ ]:
ivan = Resource(context="http://schema.org", type="Person", name="Ivan Tyron")

In [ ]:
pp(forge.as_jsonld(ivan))

In [ ]:
forge.register(ivan)

In [ ]:
pp(forge.as_jsonld(ivan, form="expanded"))

### context from the model

The configured model provides a default context that will be used to create resources that do not have context provided.

In [ ]:
hana = Resource(type="Person", name="Hana Lloyd")

In [ ]:
pp(forge.as_jsonld(hana))

In [ ]:
forge.register(hana)

In [ ]:
pp(forge.as_jsonld(hana, form="expanded"))

### context available in the store

It is possible to use a context that is only available in the Store, such as `https://bbp.neuroshapes.org`

In [ ]:
karen = Resource(context="https://bbp.neuroshapes.org", type="Person", name="Karen Jones")

In [ ]:
forge.register(karen)

In [ ]:
print(karen)

Since this context is not locally resolvable the json-ld conversion will fail.

In [ ]:
forge.as_jsonld(karen)

## JSON-LD to Resource

In [ ]:
paul = Resource(context=context, type="Person", name="Paul Owens")

In [ ]:
data = {
    "@context": context,
    "@type": "Person",
    "name": "Paul Owens",
}

In [ ]:
resource = forge.from_jsonld(data)

In [ ]:
resource == paul